# Preamble

In [1]:
%config dump.basisStateLabelingConvention="Bitstring"

"Bitstring"

In [2]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arrays;

## Amplitude Encoding

Given the repeated and padded input, prepare the qubit register state with amplitude encoding.
e.g. in a 2-qubit system, and a feature vector of $[a,b,c,d]$, $a$ maps to $\left|00\right\rangle$,
$b$ to $\left|01\right\rangle$, $c$ to $\left|10\right\rangle$, and $d$ to $\left|11\right\rangle$.

In [3]:
operation DemoAmplitudeEncoding () : Unit {
    let amplitudes = [1.0, 0.1, 2.0, 3.0];
    use qs = Qubit[2];
    let qRegister = LittleEndian(qs);
    PrepareArbitraryStateD(amplitudes, qRegister);
    
    DumpMachine();
    
    ResetAll(qs);
}

DemoAmplitudeEncoding

In [4]:
%simulate DemoAmplitudeEncoding

|00⟩	0.2671658425726324 + 0𝑖
|01⟩	0.5343316851452651 + 0𝑖
|10⟩	0.026716584257263237 + 0𝑖
|11⟩	0.8014975277178974 + 0𝑖

()

# Circuit Architecture

## Generic Parameterized Gate

In [5]:
// type representing generic gate parameters
newtype GateParams = (angle: Double, exp_beta: Double, exp_gamma: Double);

GateParams

In [6]:
/// A generic, parameterized gate
/// with the expressive power of any quantum gate
operation GenericGate (
    gateParams: GateParams,
    q: Qubit
) : Unit is Adj+Ctl {

    let angle = gateParams::angle;
    let exp_beta = gateParams::exp_beta;
    let exp_gamma = gateParams::exp_gamma;
    
    Rz(-(exp_beta + exp_gamma), q);
    Ry(-2.0 * angle, q);
    Rz(exp_gamma - exp_beta, q);
}

operation DemoGenericGate () : Unit {
    use q = Qubit();
    use cq = Qubit();
    H(cq);
    //Controlled GenericGate([cq], (GateParams(3., 1., 9.), q));
    
    GenericGate(GateParams(3., 1., 9.), q);
    
    // Rz(-(1. + 9.)) Ry(-2. * 3.) Rz(9. - 1.)
    // = Rz(-10) Ry(-6) Rz(8)
    
    DumpMachine();
    
    Reset(q);
    Reset(cq);
}

GenericGate, DemoGenericGate

In [7]:
%trace DemoGenericGate
%simulate DemoGenericGate

DisplayableHtmlElement { Html = <div id='execution-path-container-0608bbca-2f3e-4a4f-89de-9eea13121899' /> }

|00⟩	-0.37822804344190136 + -0.5890552765375272𝑖
|01⟩	-0.37822804344190136 + -0.5890552765375272𝑖
|10⟩	0.09091887768704142 + -0.04112403211672341𝑖
|11⟩	0.09091887768704142 + -0.04112403211672341𝑖

()

### Build Convolutional Circuit

In [8]:
/// Index pair representing index of a control qubit
/// and the target qubit
/// for a control gate
///
newtype ControlTargetPair = (control: Int, target: Int);

/// Type to encapsulate 1 QNN layer's parameters
///
/// ASSUMES: controlGap <= Length(theta)
///          Length(controlledTheta) <= Length(theta)
///          Length(controlledTheta) is the expected length governed by 
///
newtype LayerParams = (
    theta: GateParams[],
    controlledTheta: GateParams[],
    controlGap: Int
);

ControlTargetPair, LayerParams

In [9]:
function BuildControlTargetPairs (
    numQubits: Int,
    gapSize: Int
) : ControlTargetPair[] {
    
    mutable pairs = [];
    for idx in 1..(numQubits / GreatestCommonDivisorI(numQubits, gapSize)) {
    
        // pairs list + list containing single item, next ControlTargetPair
        set pairs +=
            [
                ControlTargetPair(
                    (idx * gapSize) % numQubits,
                    (idx * gapSize - gapSize) % numQubits
                )
            ];
    }
    
    return pairs;
}

/// Build the convolution circuit
/// theta:
///      list of GateParams
///      to be applied to GenericGates
/// controlled_theta:
///      list of GateParams
///      to be applied to Controlled GenericGates
/// control_gap:
///       the space b/w each control qubit
///       and the target qubit (w/ wrap around from qubit n-1 back to qubit 0)
///       ***larger gap means fewer control gates
///
/// ASSUMES: length of theta is equal to number of qubits in qRegister
///
operation ConvolutionalLayer (
    layerParams: LayerParams,
    qs: Qubit[]
) : Unit is Adj+Ctl {
    
    let theta = layerParams::theta;
    let controlledTheta = layerParams::controlledTheta;
    let controlGap = layerParams::controlGap;
    
    // apply the single-qubit gates
    for idx in 0..Length(qs) - 1 {
        GenericGate(theta[idx], qs[idx]);
    }
    
    // apply the controlled 2-qubit gates
    let ctlTargPairs = BuildControlTargetPairs(Length(qs), controlGap);
    for idx in 0..Length(ctlTargPairs) - 1 {
        let ctlIndex  = ctlTargPairs[idx]::control;
        let targIndex = ctlTargPairs[idx]::target;
        
        Controlled GenericGate(
            [qs[ctlIndex]],
            (controlledTheta[idx], qs[targIndex])
        );
    }
}

BuildControlTargetPairs, ConvolutionalLayer

### A Sample convolution circuit

In [11]:
operation SampleConvolutionCircuit () : Unit {
    let numQubits = 3;

    let theta = [
        GateParams(0.5, 1.7, 2.3),
        GateParams(12., 1.0, 10.),
        GateParams(7.1, 1.6, 5.3)
    ];

    let controlledTheta = [
        GateParams(1., 7., 9.),
        GateParams(4., 2., 1.),
        GateParams(3., 6., 2.)
    ];

    let controlGap = 1;
    
    let layerParams = LayerParams(theta, controlledTheta, controlGap);
    
    use qs = Qubit[numQubits];
    
    ConvolutionalLayer(layerParams, qs);
    
    ResetAll(qs);
}

SampleConvolutionCircuit

In [12]:
%trace SampleConvolutionCircuit

DisplayableHtmlElement { Html = <div id='execution-path-container-3e57b16d-4b4a-4611-bd5b-d8d63a895aca' /> }

### Sample Amplitude Encoding + Convolution Circuit + Measure

In [13]:
operation SampleFullCircuit () : Result[] {
    let numQubits = 3;
    let theta = [
        GateParams(0.5, 1.7, 2.3),
        GateParams(12., 1.0, 10.),
        GateParams(7.1, 1.6, 5.3)
    ];

    let controlledTheta = [
        GateParams(1., 7., 9.),
        GateParams(4., 2., 1.),
        GateParams(3., 6., 2.)
    ];

    let controlGap = 1;
    
    let amplitudes = [1.0, 2.0, 3.0];
    use qs = Qubit[numQubits];
    let qRegister = LittleEndian(qs);
    
    PrepareArbitraryStateD(amplitudes, qRegister);
    
    let layerParams = LayerParams(theta, controlledTheta, controlGap);
    ConvolutionalLayer(layerParams, qs);
    let results = MeasureEachZ(qs);
    
    
    ResetAll(qs);
    
    return results;
}

SampleFullCircuit

In [14]:
%trace SampleFullCircuit
%simulate SampleFullCircuit

DisplayableHtmlElement { Html = <div id='execution-path-container-aee6563b-9ef0-4c32-8145-6f6b4e370b3d' /> }

Zero, One, One

## Consider the Possibilities

### Could learn to entangle

In [15]:
operation Entangle () : Unit {
    use qs = Qubit[2];
    H(qs[0]);
    I(qs[1]);
    CNOT(qs[0], qs[1]);
    
    DumpMachine();
    
    ResetAll(qs);
}

Entangle

In [16]:
%simulate Entangle
%trace Entangle

|00⟩	0.7071067811865476 + 0𝑖
|01⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0.7071067811865476 + 0𝑖

DisplayableHtmlElement { Html = <div id='execution-path-container-d872e8fe-0145-41f2-82cf-d1ebeb394ba9' /> }

### ...Or not

In [17]:
operation DontEntangle () : Unit {
    use qs = Qubit[2];
    H(qs[1]);
    X(qs[0]);
    CNOT(qs[0], qs[1]);
    
    DumpMachine();
    
    ResetAll(qs);
}

DontEntangle

In [18]:
%simulate DontEntangle
%trace DontEntangle

|00⟩	0 + 0𝑖
|01⟩	0 + 0𝑖
|10⟩	0.7071067811865476 + 0𝑖
|11⟩	0.7071067811865476 + 0𝑖

DisplayableHtmlElement { Html = <div id='execution-path-container-743839a0-19ed-40e0-94d5-672761d116e4' /> }

## Compute Expected Value of Measurement

$$
\sigma_z = \sum_i z_i p(z_i)
$$

Approximate probabilities as frequencies over `S` attempts (`S` is a hyperparameter)

In [22]:
/// The full qnn circuit
///
/// INPUT:
///        featureVector
///            - the preprocessed feature vector
///                (
///                    with input values repeated,
///                    padded with ancillary features,
///                    and length equal to an integral factor of 2
///                )
///        allLayersParams
///            - array of each convolutional layer's params
///        sampleCount
///            - the number of samples to take when approximating state probabilities
/// ASSUMES:
///      length of every layer's theta is the same (in allLayersParams),
///      and complies with assumptions of LayerParams
///
///      
///
operation QNN (
    featureVector : Double[],
    allLayersParams : LayerParams[]
) : Int {
    
    // derive and reserve number of qubits required for amplitude encoding
    // (assumes featureVector is an integral length of a power of 2)
    let numQubits = DoubleAsInt(
        Lg(
            IntAsDouble(
                Length(featureVector)
            )
        )
    );
    
    use qs = Qubit[numQubits];
    let qRegister = LittleEndian(qs);
    
    // encode features as amplitudes
    PrepareArbitraryStateD(featureVector, qRegister);
    
    // apply the LayerParams
    for layerParams in allLayersParams {
        ConvolutionalLayer(layerParams, qs);
    }
    
    // measure
    //let results = MeasureEachZ(qs);
    let result = MeasureInteger(qRegister);
    
    // cleanup
    ResetAll(qs);
    
    // return results
    //return Mapped(x -> x == One ? true | false, results);
    return result;
}

QNN

In [25]:
operation LearnQNN (
    featureVector : Double[],
    allLayersParams : LayerParams[],
    sampleCount : Int
) : Double {
    
    mutable sum = Int(0);
    for i in 0..sampleCount {
        let outcome = QNN(featureVector, allLayersParams);
        sum += outcome;
    }
    
    return IntAsDouble(sum) / IntAsDouble(sampleCount);
}

Error QS3035: Unexpected code fragment.
 6 |     
 7 |     mutable sum = Int(0);
   |                      ^^^
 8 |     for i in 0..sampleCount {

Error QS3032: Invalid use of a reserved keyword within an expression.
 6 |     
 7 |     mutable sum = Int(0);
   |                   ^^^
 8 |     for i in 0..sampleCount {

Error QS3001: Syntax does not match any known patterns.
  9 |         let outcome = QNN(featureVector, allLayersParams);
 10 |         sum += outcome;
    |         ^^^^^^^^^^^^^^
 11 |     }

In [ ]:
operation SampleFullCircuit () : Result[] {
    let numQubits = 3;
    let theta = [
        GateParams(0.5, 1.7, 2.3),
        GateParams(12., 1.0, 10.),
        GateParams(7.1, 1.6, 5.3)
    ];

    let controlledTheta = [
        GateParams(1., 7., 9.),
        GateParams(4., 2., 1.),
        GateParams(3., 6., 2.)
    ];

    let controlGap = 1;
    
    let amplitudes = [1.0, 2.0, 3.0];
    use qs = Qubit[numQubits];
    let qRegister = LittleEndian(qs);
    
    PrepareArbitraryStateD(amplitudes, qRegister);
    
    let layerParams = LayerParams(theta, controlledTheta, controlGap);
    ConvolutionalLayer(layerParams, qs);
    let results = MeasureEachZ(qs);
    
    
    ResetAll(qs);
    
    return results;
}